In [31]:
!pip install -q sentence-transformers
!pip install -q transformers
!pip install -q datasets 
!pip install -q pandas 
!pip install -q accelerate 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [33]:
!pip uninstall -qy wandb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [34]:
train_dataset = dataset['train']
train_dataset.set_format(type='pandas')
train_df = train_dataset[:]

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201883 entries, 0 to 201882
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   english      201883 non-null  object
 1   non_english  201883 non-null  object
dtypes: object(2)
memory usage: 3.1+ MB


In [6]:
train_df.head()

,english,non_english
0,I want you to know that I believe kids will ea...,"Я хочу, щоб ви знали, що я впевнений що діти б..."
1,I want you to know that there are farmers' mar...,"Я хочу, щоб ви знали, що є фермерські ринки як..."
2,"I want you to know that me, my brother and sis...","Я хочу, щоб ви знали, що я, мої брат та сестра..."
3,I try to share this everywhere I go.,"Я намагаюся поділитися цим всюди, куди б я не ..."
4,"Not too long ago, my uncle said that he offere...","Не так давно, мій дядько сказав, що він запроп..."


In [7]:
# reset df format
train_dataset.reset_format()

In [8]:
eval_dataset = dataset['dev']
eval_dataset.set_format(type='pandas')
eval_df = eval_dataset[:]

In [9]:
eval_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 993 entries, 0 to 992
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   english      993 non-null    object
 1   non_english  993 non-null    object
dtypes: object(2)
memory usage: 15.6+ KB


In [10]:
eval_df.head()

,english,non_english
0,"Thank you so much, Chris.","Дуже дякую, Кріс!"
1,And it's truly a great honor to have the oppor...,"Справді, для мене це велика честь мати можливі..."
2,"I have been blown away by this conference, and...","Я в захваті від цієї конференції, і я хочу под..."
3,"And I say that sincerely, partly because (Mock...","І, щиро кажучи, частково тому що – (Схлипує) –..."
4,(Laughter) Put yourselves in my position.,(Сміх) Поставте себе на моє місце!


In [11]:
# reset df format
eval_dataset.reset_format()

### Load teacher and student model 

In [12]:
import torch
from sentence_transformers import SentenceTransformer
from sentence_transformers.models import Transformer, Pooling 

teacher_model_id = 'sentence-transformers/multi-qa-mpnet-base-dot-v1'
transformer_module = Transformer(teacher_model_id, model_args=dict(torch_dtype=torch.float32))
pooling_module = Pooling(
    word_embedding_dimension=transformer_module.get_word_embedding_dimension(),
    pooling_mode_cls_token=False,  # initiall True is here 
    pooling_mode_mean_tokens=True
)
teacher_model = SentenceTransformer(modules=[transformer_module, pooling_module])
teacher_model.to('cuda')
teacher_model 

2025-04-11 09:06:05.307885: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744362365.543760      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744362365.612423      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [13]:
teacher_model[0].auto_model.dtype 

torch.float32

In [14]:
import torch 
from sentence_transformers import SentenceTransformer
from sentence_transformers.models import Transformer, Pooling 

student_model_id = 'FacebookAI/xlm-roberta-base'
transformer_module = Transformer(student_model_id, model_args=dict(torch_dtype=torch.float32))
pooling_module = Pooling(
    word_embedding_dimension=transformer_module.get_word_embedding_dimension(),
    pooling_mode_cls_token=False,
    pooling_mode_mean_tokens=True
)
student_model = SentenceTransformer(modules=[transformer_module, pooling_module])
student_model.to('cuda')
student_model

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [15]:
student_model[0].auto_model.dtype 

torch.float32

In [16]:
# assert that transformer config matched for student and teacher model
assert student_model[0].get_config_dict() == teacher_model[0].get_config_dict()

### Dtaset Preprocessing

In [17]:
def prepare_dataset(batch):
    return {
        'english': batch['english'],
        'non_english': batch['non_english'],
        'label': teacher_model.encode(batch['english'], padding=True, truncation=True)
   } 

In [18]:
train_dataset_labeled = train_dataset.map(prepare_dataset, batched=True, batch_size=24000)

Map:   0%|          | 0/201883 [00:00<?, ? examples/s]

Batches:   0%|          | 0/750 [00:00<?, ?it/s]

Batches:   0%|          | 0/750 [00:00<?, ?it/s]

Batches:   0%|          | 0/750 [00:00<?, ?it/s]

Batches:   0%|          | 0/750 [00:00<?, ?it/s]

Batches:   0%|          | 0/750 [00:00<?, ?it/s]

Batches:   0%|          | 0/750 [00:00<?, ?it/s]

Batches:   0%|          | 0/750 [00:00<?, ?it/s]

Batches:   0%|          | 0/750 [00:00<?, ?it/s]

Batches:   0%|          | 0/309 [00:00<?, ?it/s]

In [19]:
train_dataset_labeled

Dataset({
    features: ['english', 'non_english', 'label'],
    num_rows: 201883
})

In [20]:
eval_dataset = eval_dataset.map(prepare_dataset, batched=True, batch_size=None)

Map:   0%|          | 0/993 [00:00<?, ? examples/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [21]:
eval_dataset

Dataset({
    features: ['english', 'non_english', 'label'],
    num_rows: 993
})

### Defining loss function and evaluators

In [22]:
import torch
from sentence_transformers.losses import MSELoss

# Computes loss between computed sentence embeddings by student ('english' and 'non_english') and target computed by teacher ('label') 
mse_loss = MSELoss(model=student_model)

In [23]:
from sentence_transformers.evaluation import MSEEvaluator, EmbeddingSimilarityEvaluator, SequentialEvaluator 


# From documentation: The MSE is computed between ||teacher.encode(source_sentences) - student.encode(target_sentences)||.
mse_eval = MSEEvaluator(
    source_sentences=eval_dataset['english'],
    target_sentences=eval_dataset['non_english'],
    name='mse-en-ua',
    teacher_model=teacher_model,
    batch_size=16
)

# STS Benchmark (Semantic Textual Similarity Benchmark)
en_en_dataset = load_dataset('mteb/sts17-crosslingual-sts', 'en-en', split='test')
en_ua_dataset = load_dataset('csv', data_files='/kaggle/input/sts-17-ua/sts17-en-ua-gpt-4o.csv', split='train') # when loading from csv by default train split is assigned
ua_ua_dataset = load_dataset('csv', data_files='/kaggle/input/sts-17-ua/sts17-ua-ua-gpt-4o.csv', split='train')  # when loading from csv by default train split is assigned

# From documentation: Evaluate a model based on the similarity of the embeddings by calculating the Spearman and Pearson rank correlation in comparison to the gold standard labels. 
en_en_eval = EmbeddingSimilarityEvaluator(
    sentences1=en_en_dataset['sentence1'],
    sentences2=en_en_dataset['sentence2'],
    scores=[score / 5.0 for score in en_en_dataset['score']],  # normalizing to score from to 1
    show_progress_bar=False,
    name='sts17-en-en',
    batch_size=16
)

en_ua_eval = EmbeddingSimilarityEvaluator(
    sentences1=en_ua_dataset['sentence1'],
    sentences2=en_ua_dataset['sentence2'],
    scores=[score / 5.0 for score in en_ua_dataset['score']],
    show_progress_bar=False,
    name='sts17-en-ua',
    batch_size=16
)

ua_ua_eval = EmbeddingSimilarityEvaluator(
    sentences1=ua_ua_dataset['sentence1'],
    sentences2=ua_ua_dataset['sentence2'],
    scores=[score / 5.0 for score in ua_ua_dataset['score']],
    show_progress_bar=False,
    name='sts17-ua-ua',
    batch_size=16
)

# Composing evaluators in one chain! 
evaluator = SequentialEvaluator([mse_eval, en_en_eval, en_ua_eval, ua_ua_eval]) 

README.md:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

en-en.jsonl.gz:   0%|          | 0.00/8.86k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/250 [00:00<?, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [24]:
from sentence_transformers import SentenceTransformerTrainingArguments

train_args = SentenceTransformerTrainingArguments(
    output_dir='./xlm-roberta-ua-distilled',
    fp16=False,
    num_train_epochs=5,
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=3,  # simulate batch of accum_steps * per_device_train_batch_size = 3 * 16 = 48
    
    eval_steps=256, 
    eval_strategy='steps',
    
    save_steps=256, 
    save_strategy='steps',
    save_total_limit=2,
    
    logging_steps=256
)

### Training!

In [25]:
from sentence_transformers import SentenceTransformerTrainer

trainer = SentenceTransformerTrainer(
    model=student_model,
    args=train_args,
    train_dataset=train_dataset_labeled,
    eval_dataset=eval_dataset,
    loss=mse_loss,
    evaluator=evaluator 
)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [26]:
import logging
logging.basicConfig(level=logging.DEBUG)

trainer.train()

Step,Training Loss,Validation Loss,Mse-en-ua Negative Mse,Sts17-en-en Pearson Cosine,Sts17-en-en Spearman Cosine,Sts17-en-ua Pearson Cosine,Sts17-en-ua Spearman Cosine,Sts17-ua-ua Pearson Cosine,Sts17-ua-ua Spearman Cosine,Sequential Score
256,0.207400,0.083567,-9.047288,0.059548,0.173122,-0.033824,-0.048771,0.019021,0.028516,0.028516
512,0.131800,0.270461,-29.518756,-0.043769,0.057961,0.099155,-0.067967,0.128295,0.274267,0.274267
768,0.114600,0.110842,-18.071407,0.175364,0.191009,-0.008102,0.007108,0.056728,0.176098,0.176098
1024,0.108500,0.118889,-20.275068,-0.038271,0.060960,0.100826,0.095551,0.217610,0.231156,0.231156
1280,0.102600,0.321619,-31.981030,0.150335,0.175102,0.167003,0.151609,0.183439,0.254498,0.254498
1536,0.096300,0.206900,-20.458837,0.205979,0.198364,-0.073896,-0.102256,0.109237,0.178505,0.178505
1792,0.093300,0.280708,-28.385133,0.076089,0.176890,-0.049274,-0.158114,0.010109,0.177335,0.177335
2048,0.091800,0.135312,-19.327693,0.272888,0.328820,-0.086076,-0.107651,0.042067,0.141469,0.141469
2304,0.090700,0.113981,-18.240812,0.346075,0.367256,0.086045,0.078545,0.045792,0.208089,0.208089
2560,0.089900,0.096401,-15.752825,0.185638,0.255947,0.217652,0.230752,0.203433,0.174880,0.174880


TrainOutput(global_step=21030, training_loss=0.07928810872410345, metrics={'train_runtime': 16466.3615, 'train_samples_per_second': 61.302, 'train_steps_per_second': 1.277, 'total_flos': 0.0, 'train_loss': 0.07928810872410345, 'epoch': 5.0})

In [27]:
trainer.evaluate()

{'eval_loss': 0.02040005289018154,
 'eval_mse-en-ua_negative_mse': -2.1519742906093597,
 'eval_sts17-en-en_pearson_cosine': 0.5273567306654277,
 'eval_sts17-en-en_spearman_cosine': 0.5630721403406446,
 'eval_sts17-en-ua_pearson_cosine': 0.36821999547054507,
 'eval_sts17-en-ua_spearman_cosine': 0.3551228317268819,
 'eval_sts17-ua-ua_pearson_cosine': 0.48312967918474736,
 'eval_sts17-ua-ua_spearman_cosine': 0.5271830395192819,
 'eval_sequential_score': 0.5271830395192819,
 'eval_runtime': 7.4068,
 'eval_samples_per_second': 134.065,
 'eval_steps_per_second': 8.506,
 'epoch': 5.0}

### Saving

In [28]:
student_model.save('./xlm-roberta-ua-distilled/final')

### Push to hub

In [37]:
import os 

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

os.environ['HF_TOKEN'] = user_secrets.get_secret("HF_TOKEN")
student_model.push_to_hub('xlm-roberta-ua-distilled')

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

'https://huggingface.co/panalexeu/xlm-roberta-ua-distilled/commit/495c39c51e6ebbd40c242d77629b8b57a70caab3'